In [8]:
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.optimize as opt
%matplotlib inline

In [7]:
base_dir = 'D:\\Src\\ensemble-forecasting'  # home
h = np.loadtxt(os.path.join(base_dir, 'data', '2011', '2011080100_hiromb_GI_60x434.txt')) - 37.356
s = np.loadtxt(os.path.join(base_dir, 'data', '2011', '2011080100_swan_GI_48x434.txt'))
n = np.loadtxt(os.path.join(base_dir, 'data', '2011', '2011080100_noswan_GI_48x434.txt'))
m = np.loadtxt(os.path.join(base_dir, 'data', '2011', '2011080100_measurements_GI_2623.txt'))

# forecast dimensions
N = 430
T = 48

# preparing measurements forecast
m_fc = np.zeros((N, T+1))
for i in range(N):
    for j in range(T+1):
        m_fc[i, j] = m[i*6+j]

# data cut and shifting
shift_const = 0
h = h[0:N, 1:T+1] + shift_const
s = s[0:N, 1:T+1] + shift_const
n = n[0:N, 1:T+1] + shift_const
m_fc = m_fc[0:N, 1:T+1] + shift_const
h.shape, s.shape, n.shape, m_fc.shape

((430, 48), (430, 48), (430, 48), (430, 48))

In [18]:
ensembles_mask = np.array([[i & 1, (i >> 1) & 1, (i >> 2) & 1] for i in range(1, 8)])
def lin_comb(src, coeff):
    res = np.full(src[0].shape, coeff[-1])
    for i in range(len(src)):
        res += src[i] * coeff[i]
def get_coeff(mask, idcs):
    srcs = [pair[0][idcs[1]:idcs[2]] for pair in zip([h, s, n], mask) if pair[1]]
    return opt.minimize(lambda k: , np.full(sum(mask) + 1, 1.0, dtype=float))

[1, 2]